In [1]:
import numpy as np
import matplotlib.pyplot as plt

from psiam_tied_dv_map_utils import psiam_tied_data_gen_wrapper_V_A_change
from joblib import Parallel, delayed
import pandas as pd

# Params

In [2]:
V_A = 1.1
theta_A = 1.8


rate_lambda = 0.14
T_0 = 0.66 * (1e-3)
theta_E = 40.5

t_A_aff = 0.03
t_E_aff = 0.05
t_motor = 0.03

Z_E = 0
L = 0.5


V_A_step = 0.2


N_sim = int(50e3)
dt = 1e-4


# ABL, ILD, stim and led times from data

In [3]:
og_df = pd.read_csv('../out_LED.csv')
df = og_df[ og_df['repeat_trial'].isin([0,2]) | og_df['repeat_trial'].isna() ]
session_type = 7    
df = df[ df['session_type'].isin([session_type]) ]
training_level = 16
df = df[ df['training_level'].isin([training_level]) ]

t_stim_and_led_tuple = [(row['intended_fix'], row['LED_onset_time']) for _, row in df.iterrows()]

ABL_arr = df['ABL'].unique()
ABL_arr.sort()

ILD_arr = df['ILD'].unique()
ILD_arr.sort()


# Simulate

In [ ]:
new_V_A = V_A + V_A_step
N_print = int(1)
sim_results =  Parallel(n_jobs=-1)(
                            delayed(psiam_tied_data_gen_wrapper_V_A_change)
                                  (V_A, theta_A, ABL_arr, ILD_arr, rate_lambda, T_0, theta_E, Z_E, t_A_aff, t_E_aff, t_motor, L, \
                                  t_stim_and_led_tuple, new_V_A, iter_num, N_print, dt)
                                    
                                    for iter_num in range(N_sim)
                                    
                                    )

# all_loglike = Parallel(n_jobs=-1)(delayed(compute_loglike)(t, V_A, theta_A, NDT, area, pdf_factor)\
#                                        for t in abort_RT if t > T_trunc)

os id: 869680, In iter_num: 0, ABL: 20, ILD: -16.0, t_stim: 0.766172920450677
os id: 869681, In iter_num: 1, ABL: 40, ILD: -2.0, t_stim: 0.425414512246301
os id: 869682, In iter_num: 2, ABL: 40, ILD: 2.0, t_stim: 0.344957686168934
os id: 869683, In iter_num: 3, ABL: 20, ILD: -8.0, t_stim: 0.32394537120617
os id: 869684, In iter_num: 4, ABL: 20, ILD: -1.0, t_stim: 0.900961051959694
os id: 869685, In iter_num: 5, ABL: 40, ILD: 2.0, t_stim: 0.514378746843967
os id: 869718, In iter_num: 6, ABL: 40, ILD: 2.0, t_stim: 0.599797844890358
os id: 869717, In iter_num: 7, ABL: 60, ILD: -2.0, t_stim: 0.730630799671329
os id: 869719, In iter_num: 8, ABL: 20, ILD: 2.0, t_stim: 0.319943266286228
os id: 869720, In iter_num: 9, ABL: 60, ILD: 1.0, t_stim: 0.74366276397834
os id: 869721, In iter_num: 10, ABL: 60, ILD: 2.0, t_stim: 0.412648445197317
os id: 869722, In iter_num: 11, ABL: 20, ILD: -8.0, t_stim: 0.282390611975537
os id: 869723, In iter_num: 12, ABL: 60, ILD: 4.0, t_stim: 0.966345906346465
os i